# Day 7
https://adventofcode.com/2018/day/7

In [1]:
import aocd
data = aocd.get_data(year=2018, day=7)

In [2]:
from dataclasses import dataclass
from typing import Set
import re

In [3]:
re_task = re.compile(r'Step (\w) must be finished before step (\w) can begin.')

In [4]:
@dataclass(frozen=True, order=True)
class Task():
    name: str
    prerequisites: Set[str]

    @property
    def time_to_complete(self):
        return ord(self.name) - 4
    
    def can_begin(self, completed):
        return all(prereq in completed for prereq in self.prerequisites)

    @classmethod
    def all_from_text(cls, text):
        prereqs = dict()
        for (prereq, step) in re_task.findall(text):
            if prereq not in prereqs:
                prereqs[prereq] = set()
            if step not in prereqs:
                prereqs[step] = set()
            prereqs[step].add(prereq)
        return sorted([Task(name, frozenset(prereqset)) for (name, prereqset) in prereqs.items()])

In [5]:
@dataclass
class Worker():
    task: Task
    time_spent: int
    
    def __init__(self):
        self.assign(None)
    
    def assign(self, task):
        self.task = task
        self.time_spent = 0

    def progress(self):
        if self.task:
            self.time_spent += 1
            if self.time_spent == self.task.time_to_complete:
                completed = self.task
                self.assign(None)
                return completed

In [6]:
def complete_all_tasks(tasklist, workercount):
    time = 0
    order = []
    workers = [Worker() for w in range(workercount)]
    
    while(len(order) < len(tasklist)):
        # assign idle workers to ready tasks
        unassigned = [worker for worker in workers if worker.task is None]
        in_progress = {worker.task for worker in workers}
        ready = sorted(task for task in tasklist
                       if task not in in_progress and task.name not in order and task.can_begin(order))
        for (worker, task) in zip(unassigned, ready):
            worker.assign(task)

        # advance time on all tasks
        time += 1
        for completed_task in [worker.progress() for worker in workers]:
            if completed_task:
                order.append(completed_task.name)

    return (time, ''.join(order))

In [7]:
tasks = Task.all_from_text(data)

time1, p1 = complete_all_tasks(tasks, 1)
print('Part 1: {}'.format(p1))

p2, order2 = complete_all_tasks(tasks, 5)
print('Part 2: {}'.format(p2))

Part 1: BGJCNLQUYIFMOEZTADKSPVXRHW
Part 2: 1017
